In [2]:
bn_path = "./nets/collection/"
from bni_netica.bni_netica import *
from bni_netica.bni_netica import Net
from bni_netica.bni_utils import findAllDConnectedNodes

CancerNeapolitanNet = Net(bn_path+"Cancer Neapolitan.neta")
ChestClinicNet = Net(bn_path+"ChestClinic.neta")
ClassifierNet = Net(bn_path+"Classifier.neta")
CoronaryRiskNet = Net(bn_path+"Coronary Risk.neta")
FireNet = Net(bn_path+"Fire.neta")
MendelGeneticsNet = Net(bn_path+"Mendel Genetics.neta")
RatsNet = Net(bn_path+"Rats.neta")
WetGrassNet = Net(bn_path+"Wet Grass.neta")
RatsNoisyOr = Net(bn_path+"Rats_NoisyOr.dne")
Derm = Net(bn_path+"Derm 7.9 A.dne")
CauseEffectNet = Net("./nets/outputs/common_cause_effect.neta")
netDir = "./nets/"
myNet = Net(netDir+"NF_V1.dne")
listOfNets = [CancerNeapolitanNet, ChestClinicNet, ClassifierNet, CoronaryRiskNet, FireNet, MendelGeneticsNet, RatsNet, WetGrassNet, RatsNoisyOr, Derm]

def printNet(net):
    for node in net.nodes():
        print(f"{node.name()} -> {[child.name() for child in node.children()]}")

def is_connected(net, node1, node2):
  relatedNodes = net.node(node1).getRelated("d_connected")
  return any(n.name() == node2 for n in relatedNodes)

def stateIdx(node, spec):
    if isinstance(spec, int):
        return spec
    st = node.state(spec)  # by state NAME
    if st is None:
        st = node.stateByTitle(spec)  # fallback by title
    if st is None:
        raise ValueError(f"Unknown state '{spec}' for node '{node.name()}'")
    return st.stateNum

def get_BN_node_states(net):
    structure = ""
    for node in net.nodes():
        states = [s.name() for s in node.states()]
        structure += f"{node.name()} {states}\n"
    return structure

Loading Netica


## Test

In [3]:
print('get_BN_node_states(myNet):', get_BN_node_states(myNet))

get_BN_node_states(myNet): Rainfall ['Below_average', 'Average', 'Above_average']
Drought ['Yes', 'No']
TreeCond ['Good', 'Damaged', 'Dead']
PesticideUse ['High', 'Low']
PesticideInRiver ['High', 'Low']
RiverFlow ['Good', 'Poor']
FishAbundance ['High', 'Medium', 'Low']



# Common Cause & Effects

In [4]:
def _names(nodes):
    return {n.name() for n in nodes}

def ancestors(net, node):
    return _names(net.node(node).getRelated("ancestors,exclude_self"))

def descendants(net, node):
    return _names(net.node(node).getRelated("descendents,exclude_self"))

def get_common_cause(net, node1, node2):
    return ancestors(net, node1) & ancestors(net, node2)

def get_common_effect(net, node1, node2):
    return descendants(net, node1) & descendants(net, node2)


## Test

In [5]:
# net
# A → B → C ← D ← E
# A ← X ← Z → Y → E

print('Common Causes of X and E:', get_common_cause(CauseEffectNet, "X", "E"))
print('Common Effects of B and E:', get_common_effect(CauseEffectNet, "B", "E"))

print('Common Causes of A and E:', get_common_cause(CauseEffectNet, "A", "E"))
print('Common Effects of A and E:', get_common_effect(CauseEffectNet, "A", "E"))

Common Causes of X and E: {'Z'}
Common Effects of B and E: {'C'}
Common Causes of A and E: {'Z'}
Common Effects of A and E: {'C'}


# Check for change of dependancy of X, Y after observing Z

In [6]:
from contextlib import contextmanager

@contextmanager # release resource automatically after using it
def _temporarily_set_findings(net, findings_dict):
    """findings_dict: {node_name: state_name_or_index}"""
    saved = net.findings()  # current evidence (indices)
    try:
        # Apply new evidence
        for k, v in findings_dict.items():
            node = net.node(k)
            if isinstance(v, int):
                node.finding(v)
            else:
                node.finding(v)  # state(name)
        net.update()
        yield
    finally:
        # Restore previous evidence
        net.retractFindings()
        for k, v in (saved or {}).items():
            net.node(k).finding(v)
        net.update()

def does_Z_change_dependency(net, X, Y, Z):
    """
    Returns (changed: bool, details: {'before': bool, 'after': bool})
    where True means 'd-connected' (dependent), False means 'd-separated' (independent).
    """
    zname = net.node(Z).name()
    saved = net.findings()  # keep current evidence E to restore later
    
    try:
        # ---- BEFORE: dependency without Z ----
        if saved and zname in saved:
            # ensure Z is not observed for the 'before' check
            net.node(zname).retractFindings()
            net.update()
        dep_before = is_connected(net, X, Y)

        # ---- AFTER: dependency under Z observed ----
        with _temporarily_set_findings(net, {zname: 0}): # using state index 0 for Z
            dep_after = is_connected(net, X, Y)

        return (dep_before != dep_after), {"before": dep_before, "after": dep_after}
    finally:
        # Restore original evidence exactly as it was
        net.retractFindings()
        if saved:
            net.findings(saved)
        net.update()

def evidences_block_XY(net, X, Y):
    ans = []
    evidences = findAllDConnectedNodes(net, X, Y)
    # print('evidences:', [e.name() for e in evidences])
    for e in evidences:
        e_name = e.name()
        if e_name != X and e_name != Y and does_Z_change_dependency(net, X, Y, e_name)[0]:
            ans.append(e_name)
    return ans

## Test

In [7]:
print('Net: ', myNet.name())
printNet(myNet)
print()

Net:  NativeFishV1
Rainfall -> ['TreeCond', 'PesticideInRiver', 'RiverFlow']
Drought -> ['TreeCond', 'RiverFlow']
TreeCond -> []
PesticideUse -> ['PesticideInRiver']
PesticideInRiver -> ['FishAbundance']
RiverFlow -> ['FishAbundance']
FishAbundance -> []



In [8]:
print('Net: ', myNet.name())
printNet(myNet)
print()

findingsPest = myNet.node("PesticideInRiver").stateNames()
print('Findings on PesticideInRiver:', findingsPest)

print()

print(evidences_block_XY(myNet, "RiverFlow", "PesticideInRiver"))
# print(does_Z_affect_XY(myNet, X="Rainfall", Y="RiverFlow", Z="FishAbundance"))

# print('does_Z_change_dependency:', end=' ')
print(does_Z_change_dependency(myNet, X="Rainfall", Y="Drought", Z="TreeCond"))


Net:  NativeFishV1
Rainfall -> ['TreeCond', 'PesticideInRiver', 'RiverFlow']
Drought -> ['TreeCond', 'RiverFlow']
TreeCond -> []
PesticideUse -> ['PesticideInRiver']
PesticideInRiver -> ['FishAbundance']
RiverFlow -> ['FishAbundance']
FishAbundance -> []

Findings on PesticideInRiver: ['High', 'Low']

['Rainfall']
(True, {'before': False, 'after': True})


# Prob of X when observe Y

In [9]:
def _output_distribution(beliefs, node):
    """Pretty print a distribution (list of probabilities) with state names."""
    output = ""
    for i, p in enumerate(beliefs):
        state = node.state(i)
        output += f"  P({node.name()}={state.name()}) = {p:.4f}\n"

    return output

def prob_X_given_Y(net, X=None, Y=None, y_state="Yes"):
    """
    Returns P(X | Y = y_state), net_after_observation
    y_state can be state names (str) or indices (int).
    """
    
    with _temporarily_set_findings(net, {Y: y_state}):
        node_X = net.node(X)
        # beliefs() is P(X | current findings)
        return node_X.beliefs(), net
    
def prob_X_given_YZ(net, X=None, Y=None, y_state="Yes", Z=None, z_state="Yes"):
    """
    Returns P(X = x_state | Y = y_state, Z = z_state), net_after_observation
    x_state, y_state and z_state can be state names (str) or indices (int).
    """
    
    with _temporarily_set_findings(net, {Y: y_state, Z: z_state}):
        node_X = net.node(X)
        # beliefs() is P(X | current findings)
        return node_X.beliefs(), net
    
def get_prob_X_given_Y(net, X=None, Y=None, y_state="Yes"):
    """
    Returns string output of prob_X_given_Y
    """
    beliefs, net_after = prob_X_given_Y(net, X, Y, y_state)
    output = f"P({X} | {Y}={y_state}):\n"
    output += _output_distribution(beliefs, net_after.node(X))
    return output, net_after

def get_prob_X_given_YZ(net, X=None, Y=None, y_state="Yes", Z=None, z_state="Yes"):
    """
    Returns string output of prob_X_given_YZ
    """
    beliefs, net_after = prob_X_given_YZ(net, X, Y, y_state, Z, z_state)
    output = f"P({X} | {Y}={y_state}, {Z}={z_state}):\n"
    output += _output_distribution(beliefs, net_after.node(X))
    return output, net_after

## Test

In [10]:
out, _ = get_prob_X_given_Y(myNet, X="Rainfall", Y="TreeCond", y_state="Good")
print(out)

out2, _ = get_prob_X_given_YZ(myNet, X="Rainfall", Y="Drought", y_state="Yes", Z="TreeCond", z_state="Good")
print(out2)

P(Rainfall | TreeCond=Good):
  P(Rainfall=Below_average) = 0.0857
  P(Rainfall=Average) = 0.6909
  P(Rainfall=Above_average) = 0.2235

P(Rainfall | Drought=Yes, TreeCond=Good):
  P(Rainfall=Below_average) = 0.0784
  P(Rainfall=Average) = 0.6863
  P(Rainfall=Above_average) = 0.2353



# Detect Relationship

In [11]:
import math
from itertools import product

# helpers
def _to01(x):
    if isinstance(x, str):
        return 1 if x.lower().startswith('t') else 0
    return int(bool(x))

def _rmse(obs, pred):
    return math.sqrt(sum((obs[k]-pred[k])**2 for k in obs)/len(obs))

def _clip01(x): 
    return 0.0 if x < 0 else 1.0 if x > 1 else float(x)

def _ensure_keys(d):
    # normalize keys to (a,b) with 0/1
    norm = {}
    for k,v in d.items():
        a,b = k
        norm[(_to01(a), _to01(b))] = float(v)
    # fill missing (shouldn't happen)
    for a,b in product([0,1],[0,1]):
        norm.setdefault((a,b), 0.0)
    return norm

# prototypes (deterministic)
def _logical_or():
    return {(1,1):1.0,(1,0):1.0,(0,1):1.0,(0,0):0.0}

def _logical_and():
    return {(1,1):1.0,(1,0):0.0,(0,1):0.0,(0,0):0.0}

def _logical_xor():
    # 1 iff exactly one true
    return {(1,1):0.0,(1,0):1.0,(0,1):1.0,(0,0):0.0}

def _logical_xnor():
    # 1 iff both equal (often mislabeled "XOR" in sheets)
    return {(1,1):1.0,(1,0):0.0,(0,1):0.0,(0,0):1.0}

# parametric: Noisy-OR (with leak)
# P11 = 1 - (1-l)(1-pA)(1-pB);  P10 = 1-(1-l)(1-pA); P01 = 1-(1-l)(1-pB); P00=l
def _fit_noisy_or(obs):
    l = obs[(0,0)]
    denom = 1 - l
    if denom <= 0:  # degenerate: l>=1 -> always 1
        pred = {(a,b): 1.0 for a,b in obs}
        params = {"leak": l, "pA": 1.0, "pB": 1.0}
        return _rmse(obs, pred), pred, params
    pA = 1.0 - (1.0 - obs[(1,0)]) / denom
    pB = 1.0 - (1.0 - obs[(0,1)]) / denom
    # constrain to [0,1] so we don't blow up badly
    pA, pB = _clip01(pA), _clip01(pB)
    def f(a,b):
        return 1.0 - (1.0 - l) * (1.0 - pA)**a * (1.0 - pB)**b
    pred = {(a,b): f(a,b) for a,b in obs}
    return _rmse(obs, pred), pred, {"leak": l, "pA": pA, "pB": pB}

# parametric: "Noisy-AND" (simple leaky multiplicative)
# P(a,b) = l * (qA^a) * (qB^b); so P00=l, P10=l*qA, P01=l*qB, P11=l*qA*qB
def _fit_noisy_and(obs):
    l = obs[(0,0)]
    if l <= 0:
        # degenerate: l==0 -> model predicts zeros except maybe 0/0 division
        pred = {(a,b): 0.0 for a,b in obs}
        return _rmse(obs, pred), pred, {"leak": l, "qA": 0.0, "qB": 0.0}
    qA = obs[(1,0)] / l
    qB = obs[(0,1)] / l
    qA, qB = _clip01(qA), _clip01(qB)
    def f(a,b):
        return l * (qA**a) * (qB**b)
    pred = {(a,b): f(a,b) for a,b in obs}
    return _rmse(obs, pred), pred, {"leak": l, "qA": qA, "qB": qB}

# parametric: Additive (linear, clipped)
# P(a,b) = clip(bias + wA*a + wB*b)
def _fit_additive(obs):
    # Solve least squares for [bias, wA, wB]
    # X rows: (1, a, b)
    import numpy as np
    X = []
    y = []
    ordering = [(0,0),(1,0),(0,1),(1,1)]
    for a,b in ordering:
        X.append([1.0, float(a), float(b)])
        y.append(obs[(a,b)])
    X = np.asarray(X); y = np.asarray(y)
    beta, *_ = np.linalg.lstsq(X, y, rcond=None)  # [bias, wA, wB]
    def f(a,b):
        return _clip01(beta[0] + beta[1]*a + beta[2]*b)
    pred = {(a,b): f(a,b) for a,b in obs}
    return _rmse(obs, pred), pred, {"bias": float(beta[0]), "wA": float(beta[1]), "wB": float(beta[2])}

# main
def detect_relationship(cpt_Pz1, tol=1e-6):
    """
    cpt_Pz1: dict with keys like ('T','F'), ('F','T'), etc. or (1,0)
             values are P(Z=1 | A=a, B=b).
    Returns dict: {label: {'rmse':..., 'pred':..., 'params':...}}, plus 'best'.
    """
    obs = _ensure_keys(cpt_Pz1)

    # logicals
    models = {}
    for label, proto in [
        ("Logical OR", _logical_or()),
        ("Logical AND", _logical_and()),
        ("Logical XOR", _logical_xor()),
        ("Logical XNOR", _logical_xnor()),
    ]:
        rmse = _rmse(obs, proto)
        models[label] = {"rmse": rmse, "pred": proto, "params": None}

    # parametrics
    rmse_noisy_or, pred_noisy_or, par_noisy_or = _fit_noisy_or(obs)
    models["Noisy OR"] = {"rmse": rmse_noisy_or, "pred": pred_noisy_or, "params": par_noisy_or}

    rmse_noisy_and, pred_noisy_and, par_noisy_and = _fit_noisy_and(obs)
    models["Noisy AND"] = {"rmse": rmse_noisy_and, "pred": pred_noisy_and, "params": par_noisy_and}

    try:
        rmse_add, pred_add, par_add = _fit_additive(obs)
        models["Additive"] = {"rmse": rmse_add, "pred": pred_add, "params": par_add}
    except Exception:
        # numpy not available -> skip additive
        pass

    # pick best
    best_label = min(models.keys(), key=lambda k: models[k]["rmse"])
    models["best"] = best_label
    return models

# Logical OR (deterministic)
logical_or = {('T','T'):1, ('T','F'):1, ('F','T'):1, ('F','F'):0}

# “Noisy OR” example from your first sheet
noisy_or = {('T','T'):0.7, ('T','F'):0.6, ('F','T'):0.5, ('F','F'):0.0}

# Logical AND (deterministic)
logical_and = {('T','T'):1, ('T','F'):0, ('F','T'):0, ('F','F'):0}

# “Noisy AND” toy (monotone, biggest at TT)
noisy_and = {('T','T'):0.7, ('T','F'):0.6, ('F','T'):0.5, ('F','F'):0.0}

# Logical XOR / XNOR examples
logical_xor  = {('T','T'):0, ('T','F'):1, ('F','T'):1, ('F','F'):0}
logical_xnor = {('T','T'):1, ('T','F'):0, ('F','T'):0, ('F','F'):1}

# Additive sheet example
additive = {('T','T'):0.7, ('T','F'):0.6, ('F','T'):0.5, ('F','F'):0.0}

for name, tbl in [
    ("Logical OR", logical_or),
    ("Noisy OR", noisy_or),
    ("Logical AND", logical_and),
    ("Noisy AND (toy)", noisy_and),
    ("Logical XOR", logical_xor),
    ("Logical XNOR", logical_xnor),
    ("Additive", additive),
]:
    res = detect_relationship(tbl)
    print(name, "→ best:", res["best"], "rmse:", round(res[res["best"]]["rmse"], 6))

Logical OR → best: Logical OR rmse: 0.0
Noisy OR → best: Noisy OR rmse: 0.05
Logical AND → best: Logical AND rmse: 0.0
Noisy AND (toy) → best: Noisy OR rmse: 0.05
Logical XOR → best: Logical XOR rmse: 0.0
Logical XNOR → best: Logical XNOR rmse: 0.0
Additive → best: Noisy OR rmse: 0.05


In [12]:
from contextlib import contextmanager

@contextmanager # release resource automatically after using it
def _temporarily_set_findings(net, findings_dict):
    """findings_dict: {node_name: state_name_or_index}"""
    saved = net.findings()  # current evidence (indices)
    try:
        # Apply new evidence
        for k, v in findings_dict.items():
            node = net.node(k)
            if isinstance(v, int):
                node.finding(v)
            else:
                node.finding(v)  # state(name)
        net.update()
        yield
    finally:
        # Restore previous evidence
        net.retractFindings()
        for k, v in (saved or {}).items():
            net.node(k).finding(v)
        net.update()

from itertools import product

def _resolve_state_index(node, spec, use_title=False):
    """
    Resolve a state spec (int or str) to an index for a given node.
    If use_title=True, string is matched against state titles first, then names.
    """
    if isinstance(spec, int):
        return spec
    if use_title:
        st = node.stateByTitle(spec) or node.state(spec)
    else:
        st = node.state(spec) or node.stateByTitle(spec)
    if st is None:
        raise ValueError(f"Unknown state '{spec}' for node '{node.name()}'")
    return st.stateNum

def _state_names_by_indices(node, idxs):
    names = []
    for i in idxs:
        st = node.state(i)
        if st is None:
            raise ValueError(f"Bad state index {i} for node '{node.name()}'")
        names.append(st.name())
    return names

def cpt_Pequals_from_bn(
    net,
    child_name,
    parent_names,
    child_state=None,                # str or int (required if child has >1 state)
    parent_state_sets=None,          # dict: {parent_name: [state specs (str or int), ...]}
    iterate="all",                   # "all" | "first_two" | "given"
    use_titles=False                 # match strings against titles first if True
):
    """
    Build a table of P(child = child_state | parent assignments), without assuming 'true/false'.

    Returns:
      dict mapping tuple of parent state NAMES (in the same order as parent_names)
      to probability of the specified child state.
      e.g. {('given','high'): 0.63, ('not given','low'): 0.12, ...}
    """
    child = net.node(child_name)
    if child is None:
        raise ValueError(f"Child node '{child_name}' not found")

    # Resolve which child state to measure
    if child_state is None:
        # default: first state (index 0)
        child_idx = 0
    else:
        child_idx = _resolve_state_index(child, child_state, use_title=use_titles)

    # For each parent, decide which state indices to iterate
    parent_state_idxs = []
    parent_state_names = []  # parallel names for pretty keys
    for p_name in parent_names:
        p_node = net.node(p_name)
        if p_node is None:
            raise ValueError(f"Parent node '{p_name}' not found")

        if iterate == "given":
            if not parent_state_sets or p_name not in parent_state_sets:
                raise ValueError(f"Provide parent_state_sets[{p_name}] when iterate='given'")
            idxs = [_resolve_state_index(p_node, s, use_title=use_titles) for s in parent_state_sets[p_name]]

        elif iterate == "first_two":
            # take first two states (works for many binary-ish nodes)
            idxs = [0, 1] if p_node.numberStates() >= 2 else [0]

        else:  # "all"
            idxs = list(range(p_node.numberStates()))

        parent_state_idxs.append(idxs)
        parent_state_names.append(_state_names_by_indices(p_node, idxs))

    # Iterate Cartesian product of selected parent states
    table = {}
    for combo_idxs in product(*parent_state_idxs):
        # Build findings dict using indices (your ctx manager accepts int or str)
        evid = {p: idx for p, idx in zip(parent_names, combo_idxs)}

        with _temporarily_set_findings(net, evid):
            prob = child.beliefs()[child_idx]

        # Pretty key using parent state NAMES
        key = tuple(names[i_pos] for names, i_pos in zip(parent_state_names, [idxs.index(i) for idxs, i in zip(parent_state_idxs, combo_idxs)]))
        # But the above maps to relative positions; instead, resolve absolute names cleanly:
        key = tuple(net.node(p).state(i).name() for p, i in zip(parent_names, combo_idxs))

        table[key] = prob

    return table

def relabel_to01(tbl, parent_names, positives):
    # positives: {'Ecstazine': 'given', 'Neurofill': 'high'}
    p0, p1 = parent_names
    new = {}
    for (a_lbl, b_lbl), v in tbl.items():
        a = 1 if a_lbl == positives[p0] else 0
        b = 1 if b_lbl == positives[p1] else 0
        new[(a, b)] = float(v)
    return new

def get_BN_node_states(net):
    structure = ""
    for node in net.nodes():
        states = [s.name() for s in node.states()]
        structure += f"{node.name()} {states}\n"
    return structure

printNet(RatsNet)
print(get_BN_node_states(RatsNet))

tbl = cpt_Pequals_from_bn(
    RatsNet, "Social_Activity", ["Ecstazine","Neurofill"],
    child_state="high",
    iterate="given",
    parent_state_sets={"Ecstazine":["given","not_given"], "Neurofill":["high","low"]}
)
tbl01 = relabel_to01(tbl, ["Ecstazine","Neurofill"], {"Ecstazine":"given","Neurofill":"high"})
res = detect_relationship(tbl01)
print("RatsNet →", res["best"], res[res["best"]]["params"])

tbl = cpt_Pequals_from_bn(
    RatsNoisyOr, "Social_Activity", ["Ecstazine","Neurofill"],
    child_state="high",
    iterate="given",
    parent_state_sets={"Ecstazine":["given","not_given"], "Neurofill":["high","low"]}
)
tbl01 = relabel_to01(tbl, ["Ecstazine","Neurofill"], {"Ecstazine":"given","Neurofill":"high"})
res = detect_relationship(tbl01)
print("RatsNoisyOr →", res["best"], res[res["best"]]["params"])

q1Net = Net('nets/Q1-AccessmentLab.neta')
# --- extract the CPT slice for FP=True ---
tbl = cpt_Pequals_from_bn(
    q1Net,                      # your network
    child_name="FP",
    parent_names=["Trav", "Fraud"],
    child_state="True",        # child state of interest
    iterate="given",
    parent_state_sets={
        "Trav":  ["True", "False"],   # order matters: positive first
        "Fraud": ["True", "False"]
    }
)
print("Raw table:", tbl)  # e.g. {('True','True'): p11, ('True','False'): p10, ...}

# --- map to binary keys (1 for the positive label you chose above) ---
tbl01 = relabel_to01(
    tbl,
    parent_names=["Trav", "Fraud"],
    positives={"Trav": "True", "Fraud": "True"}
)

# --- detect relationship ---
res = detect_relationship(tbl01)
best = res["best"]
print("q1Net (Trav,Fraud) → FP=True:", best, res[best]["params"])


Ecstazine -> ['Neurofill', 'Social_Activity']
Neurofill -> ['Social_Activity']
Social_Activity -> ['Squeaking']
Squeaking -> []
Ecstazine ['given', 'not_given']
Neurofill ['high', 'low']
Social_Activity ['high', 'low']
Squeaking ['short', 'long']

RatsNet → Noisy OR {'leak': 0.09999998658895493, 'pA': 0.5999999238385105, 'pB': 0.7999999950329464}
RatsNoisyOr → Noisy OR {'leak': 0.09999998658895493, 'pA': 0.5999999238385105, 'pB': 0.7999999950329464}
Raw table: {('True', 'True'): 0.8999999761581421, ('True', 'False'): 0.8999999761581421, ('False', 'True'): 0.10000000894069672, ('False', 'False'): 0.009999999776482582}
q1Net (Trav,Fraud) → FP=True: Noisy OR {'leak': 0.009999999776482582, 'pA': 0.8989898749300198, 'pB': 0.09090910014534781}


# Build Net

In [13]:
import random
import string
from math import comb
from bni_netica.bni_netica import Net

def _dirichlet(alphas):
    # Dirichlet without numpy: sample Gammas and normalize
    # alpha = 1.0 → uniform random distribution. values spread across [0,1], sometimes 0.9/0.1, sometimes 0.5/0.5
    # alpha < 1.0 → sparse, more extreme values (e.g. 0.99/0.01)
    # alpha > 1.0 → dense, more balanced values (e.g. 0.6/0.4)
    import random
    vals = [random.gammavariate(a, 1.0) for a in alphas]
    s = sum(vals) or 1.0
    return [v / s for v in vals]

def build_random_bn(
    n_nodes=8,
    n_vstructures=None,      # None -> auto
    n_common_causes=None,    # None -> auto
    n_chains=None,           # None -> auto
    name="RandomNet",
    seed=None,
    states=("False", "True"),
    cpt_mode="random",       # "random" or "uniform"
    dirichlet_alpha=1.0,     # used when cpt_mode="random"
    extra_edge_prob=0.0,     # optional sparse extra edges (still acyclic) - Makes the graph denser => 1 mean DAG = maximum density
    save_path=None
):
    """
    Returns: (net, node_names, edges)
      - net: compiled Netica network
      - node_names: list of node names (A,B,...)
      - edges: set of (parent, child) tuples
    """
    if seed is not None:
        random.seed(seed)

    # ----- node names A, B, C ... then A1, B1 ... -----
    def make_names(k):
        if k <= 26:
            return list(string.ascii_uppercase[:k])
        base = list(string.ascii_uppercase)
        out, idx = [], 0
        while len(out) < k:
            suffix = "" if idx == 0 else str(idx)
            for ch in base:
                out.append(f"{ch}{suffix}")
                if len(out) == k:
                    break
            idx += 1
        return out

    node_names = make_names(n_nodes)

    # ----- start an empty net & add nodes (all with same states) -----
    net = Net()
    net.name(name)
    for nm in node_names:
        net.addNode(nm, states=list(states))

    # ----- pick a random topological order; forward edges only -> DAG -----
    topo = node_names[:]
    random.shuffle(topo)
    pos = {nm: i for i, nm in enumerate(topo)}

    edges = set()

    def add_edge(u, v):
        if u == v:
            return False
        if (u, v) in edges:
            return False
        if pos[u] >= pos[v]:  # only forward in topo order
            return False
        edges.add((u, v))
        return True

    # ----- auto-counts for motif requests -----
    # The number of distinct ordered triples i<j<k is C(n,3).
    # We cap each motif type well below that to avoid over-dense graphs.
    max_triples = comb(n_nodes, 3) if n_nodes >= 3 else 0

    def auto_count(requested, density=0.25):
        if requested is None:
            cap = int(max_triples * density)
            return random.randint(0, cap) if cap > 0 else 0
        return min(int(requested), max_triples)

    n_vstructures = auto_count(n_vstructures, density=0.25)
    n_common_causes = auto_count(n_common_causes, density=0.25)
    n_chains = auto_count(n_chains, density=0.25)

    # helper to sample a triple consistent with topo order (i<j<k)
    def sample_triple():
        i, j, k = sorted(random.sample(range(n_nodes), 3))
        return topo[i], topo[j], topo[k]  # earliest, middle, latest

    # ---- place v-structures: A -> C <- B (A<C, B<C) ----
    attempts, placed = 0, 0
    MAX_TRIES = 50 * max(1, n_vstructures)
    while placed < n_vstructures and attempts < MAX_TRIES:
        attempts += 1
        A, B, C = sample_triple()   # A<B<C in order
        ok = add_edge(A, C) & add_edge(B, C)
        if ok:
            placed += 1

    # ---- place common causes: C -> A and C -> B (C<A, C<B) ----
    attempts, placed = 0, 0
    MAX_TRIES = 50 * max(1, n_common_causes)
    while placed < n_common_causes and attempts < MAX_TRIES:
        attempts += 1
        C, A, B = sample_triple()   # C<A<B
        ok = add_edge(C, A) & add_edge(C, B)
        if ok:
            placed += 1

    # ---- place chains: A -> B -> C (A<B<C) ----
    attempts, placed = 0, 0
    MAX_TRIES = 50 * max(1, n_chains)
    while placed < n_chains and attempts < MAX_TRIES:
        attempts += 1
        A, B, C = sample_triple()   # A<B<C
        ok = add_edge(A, B)
        ok &= add_edge(B, C)
        if ok:
            placed += 1

    # optional sprinkle of extra forward edges (keeps DAG)
    if extra_edge_prob > 0:
        for i, u in enumerate(topo):
            for v in topo[i+1:]:
                if random.random() < extra_edge_prob:
                    add_edge(u, v)

    # ---- materialize edges in the Netica net ----
    for (u, v) in edges:
        net.node(u).addChildren([v])

    # ---- initialize CPTs ----
    # "uniform": each row uniform
    # "random": each row ~ Dirichlet(alpha,...,alpha)
    for n in net.nodes():
        s = n.numberStates()
        parents = n.parents()
        num_rows = 1
        for p in parents:
            num_rows *= p.numberStates()

        rows = []
        for _ in range(num_rows):
            if cpt_mode == "uniform":
                rows.append([1.0 / s] * s)
            else:
                rows.append(_dirichlet([dirichlet_alpha] * s))
        n.cpt(rows)
        n.experience(1)

    # compile + save
    net.compile()
    save_path = save_path + f"{name}.dne"
    net.write(save_path)

    return net, node_names, edges

In [14]:
output_path = "./nets/outputs/"

# 1) Everything auto, random CPTs, saved to AutoBN.dne
net, nodes, edges = build_random_bn(
    n_nodes=10,
    name="AutoBN",
    seed=7,
    cpt_mode="random",
    dirichlet_alpha=1.0,
    save_path=output_path
)

# 2) Explicit motif counts, uniform CPTs (i.e., 50/50 for binary)
net2, nodes2, edges2 = build_random_bn(
    n_nodes=8,
    n_vstructures=3,
    n_common_causes=2,
    n_chains=1,
    name="UniformBN",
    cpt_mode="uniform",
    save_path=output_path
)

# 3) Heavier randomization (skewed rows) by lowering alpha
net3, nodes3, edges3 = build_random_bn(
    n_nodes=12,
    n_vstructures=None,        # auto
    n_common_causes=None,      # auto
    n_chains=None,             # auto
    name="SkewBN",
    cpt_mode="random",
    dirichlet_alpha=0.3,       # more spiky probabilities
    extra_edge_prob=0.05,
    save_path=output_path
)

## Test the boundary

In [17]:
net2, nodes2, edges2 = build_random_bn(
    n_nodes=40,
    n_vstructures=3,
    n_common_causes=2,
    n_chains=1,
    name="FortyRandomBN",
    cpt_mode="random",
    extra_edge_prob=1.0/(40-1),  # approx 1 extra edge per node on average
    save_path=output_path
)

#### Archive codes

In [16]:
# def joint_prob(net, assignments):
#     """
#     assignments: dict { node_name: state_index or state_name }
#                  (state_name must exist on that node)
#     Returns P(assignments | current findings)
#     """
#     # Build a stable order
#     items = list(assignments.items())

#     # Make node list
#     node_list = g.NewNodeList2_bn(len(items), net.eNet)

#     # States array (Netica 'state_bn' = c_int)
#     states = (state_bn * len(items))()

#     for i, (name, val) in enumerate(items):
#         node = net.node(name)
#         if node is None:
#             raise ValueError(f"Unknown node '{name}'")

#         # Put node pointer into nodelist
#         g.SetNthNode_bn(node_list, i, node.eId)

#         # Resolve state index
#         if isinstance(val, int):
#             s_idx = val
#         else:
#             st = node.state(val)
#             if st is None:
#                 raise ValueError(f"Unknown state '{val}' on node '{name}'")
#             s_idx = st.stateNum

#         states[i] = s_idx

#     # Make sure beliefs are up to date
#     net.update()

#     # Call Netica
#     prob = g.JointProbability_bn(node_list, states)
#     return float(prob)


# def xy_joint_table_by_Z(net, X, Y, Z):
#     nodeZ = net.node(Z)
#     table = {}
#     for zi in range(nodeZ.numberStates()):
#         zname = nodeZ.state(zi).name()
#         with temporarily_set_findings(net, {Z: zi}):
#             # Example: return the joint table for X×Y under Z=zi
#             nodeX, nodeY = net.node(X), net.node(Y)
#             j = {}
#             for xi in range(nodeX.numberStates()):
#                 for yi in range(nodeY.numberStates()):
#                     p = joint_prob(net, {X: xi, Y: yi})
#                     j[(nodeX.state(xi).name(), nodeY.state(yi).name())] = p
#             table[zname] = j
#     return table

# def does_Z_affect_XY(net, X, Y, Z, tol=1e-12):
#     # the result table (a Python dict) that stores all the joint probabilities you computed for each value of Z.
#     # dict { z_state_name: { (x_state_name, y_state_name): prob, ... }, ... }
#     tbl = xy_joint_table_by_Z(net, X, Y, Z) 
    
#     # Compare the first table to the others
#     it = iter(tbl.values())
#     first = next(it)
#     for other in it:
#         for k in first:
#             if abs(first[k] - other[k]) > tol:
#                 return True, tbl  # differs for at least one state of Z
#     return False, tbl

# relatedToPesticideUse = myNet.node("PesticideInRiver").parents()
# print('parents of PesticideInRiver:', [n.name() for n in relatedToPesticideUse])
# set1 = set(n.name() for n in relatedToPesticideUse)

# relatedToPesticideUse = myNet.node("RiverFlow").parents()
# print('parents of RiverFlow:', [n.name() for n in relatedToPesticideUse])
# set2 = set(n.name() for n in relatedToPesticideUse)

# print('Intersection:', set1.intersection(set2))

# relatedToPesticideUse = myNet.node("PesticideInRiver").getRelated("d_connected")
# print('relatedTo PesticideInRiver:', [n.name() for n in relatedToPesticideUse])
# set1 = set(n.name() for n in relatedToPesticideUse)

# relatedToPesticideUse = myNet.node("RiverFlow").getRelated("d_connected")
# print('relatedTo RiverFlow:', [n.name() for n in relatedToPesticideUse])
# set2 = set(n.name() for n in relatedToPesticideUse)

# print('Intersection:', set1.intersection(set2))

# print()

# print('[d-connected nodes between PesticideInRiver and RiverFlow]')
# nodes = bni_utils.findAllDConnectedNodes(myNet, "PesticideInRiver", "RiverFlow")
# print([n.name() for n in nodes])

# print('[d-connected nodes between PesticideUse and Rainfall]')

# nodes = bni_utils.findAllDConnectedNodes(myNet, "PesticideUse", "Rainfall")
# print([n.name() for n in nodes])

# print('[d-connected nodes between PesticideUse and FishAbundance]')
# nodes = bni_utils.findAllDConnectedNodes(myNet, "PesticideUse", "FishAbundance")
# print([n.name() for n in nodes])

# print('[d-connected nodes between PesticideUse and Rainfall | PesticideInRiver]')
# myNet.node('PesticideInRiver').finding('Low')
# nodes = bni_utils.findAllDConnectedNodes(myNet, "PesticideUse", "Rainfall")
# print([n.name() for n in nodes])
# myNet.retractFindings()

# print('[d-connected nodes between PesticideUse and Rainfall | FishAbundance]')
# myNet.node('FishAbundance').finding('Low')
# arcs = bni_utils.findAllDConnectedNodes(myNet, "PesticideUse", "Rainfall", {"arcs": True})
# print([arc for arc in arcs])